<a href="https://colab.research.google.com/github/sherna90/matematicas_discretas/blob/master/algoritmo_RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!pip install cryptography

In [53]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

private_key = rsa.generate_private_key(public_exponent=65537,key_size=1024,backend=default_backend())
# Extract the public key from the private key.
public_key = private_key.public_key()

private_key_bytes = private_key.private_bytes(encoding=serialization.Encoding.PEM,
                                              format=serialization.PrivateFormat.TraditionalOpenSSL,encryption_algorithm=serialization.NoEncryption())

# Convert the public key into bytes.
public_key_bytes = public_key.public_bytes(encoding=serialization.Encoding.PEM,format=serialization.PublicFormat.SubjectPublicKeyInfo)

# Convert the private key bytes back to a key.
# Because there is no encryption of the key, there is no password.
private_key = serialization.load_pem_private_key(private_key_bytes,backend=default_backend(),password=None)


public_key = serialization.load_pem_public_key(public_key_bytes,backend=default_backend())

In [54]:
public_key.public_numbers()

<RSAPublicNumbers(e=65537, n=140326996033992949762289809743712776673528562712955861685876993400349635911875941410571039356193528526896014600010091341313501095688467918420320916980766049688382382042094565782376857910531526611706925789100696323029217933422783353513667348328786142158348047150713863167589948117247549679874301179094844351681)>

In [55]:
e=public_key.public_numbers().e

In [66]:
n=private_key.private_numbers().public_numbers.n

In [67]:
n

140326996033992949762289809743712776673528562712955861685876993400349635911875941410571039356193528526896014600010091341313501095688467918420320916980766049688382382042094565782376857910531526611706925789100696323029217933422783353513667348328786142158348047150713863167589948117247549679874301179094844351681

In [68]:
public_key.public_numbers().n

140326996033992949762289809743712776673528562712955861685876993400349635911875941410571039356193528526896014600010091341313501095688467918420320916980766049688382382042094565782376857910531526611706925789100696323029217933422783353513667348328786142158348047150713863167589948117247549679874301179094844351681

In [61]:
def gcdExtended(a, b):
 
    # Base Case
    if a == 0 : 
        return b, 0, 1
            
    gcd, x1, y1 = gcdExtended(b%a, a)
    
    # Update x and y using results of recursive
    # call
    x = y1 - (b//a) * x1
    y = x1
    
    return gcd, x, y

gcdExtended(e,d)

(1,
 -12948643599354606882655366782942176201502558342710557113790406738949892011417192961606302961427480517546163035450506696902026378269644485726320769743825943512557538812650931280891091675554808320878675813510378611154575963212895212327120803580255612842519388507055109569813687807736178382297933758411429757558,
 19319)

In [69]:
def powmod(a: int, e: int, n: int) -> int:
    accum: int = 1
    apow2: int = a
    while e > 0:
        if e & 1:
            accum = (accum * apow2) % n
        apow2 = (apow2 * apow2) % n
        e = e >> 1
    return accum

In [70]:
def simple_rsa_encrypt(m, publickey):
  numbers = publickey.public_numbers()
  return powmod(m, numbers.e, numbers.n)

def simple_rsa_decrypt(c, privatekey):
  numbers = privatekey.private_numbers()
  return powmod(c, numbers.d, numbers.public_numbers.n)

def int_to_bytes(i):
  i = int(i)
  return i.to_bytes((i.bit_length()+7)//8, byteorder='big')

def bytes_to_int(b):
  return int.from_bytes(b, byteorder='big')

In [38]:
CAP=3116

In [72]:
int_to_bytes(CAP)

b'\x0c,'

In [73]:
bytes_to_int(b'\x0c,')

3116

In [74]:
arr = str.encode('CAP','ascii')

In [75]:
bytes_to_int(arr)

4407632

In [78]:
C=simple_rsa_encrypt(bytes_to_int(arr),public_key)

In [79]:
C

93530776273134146960409741983940802848275208766161080587466151784010787585093914525967124407875692002393052265018694537463329335795216250105843463546607479150481916071175279312653991290219586668785665555355575452920028311128805495848932607165521656751314737341177404141711615355833606445706309086821252880806

In [82]:
M=simple_rsa_decrypt(C, private_key)

In [83]:
M

4407632

In [84]:
int_to_bytes(M)

b'CAP'